In [8]:
import importlib
import json
import sys
import typing as ty
from pathlib import Path
from tempfile import TemporaryDirectory

from datamodel_code_generator import DataModelType, InputFileType, generate
from pydantic import BaseModel


def pydantic_model_from_json_schema(json_schema: str) -> ty.Type[BaseModel]:
    load = json_schema["title"] if "title" in json_schema else "Model"

    with TemporaryDirectory() as temporary_directory_name:
        temporary_directory = Path(temporary_directory_name)
        file_path = "model.py"
        module_name = file_path.split(".")[0]
        output = Path(temporary_directory / file_path)
        generate(
            json.dumps(json_schema),
            input_file_type=InputFileType.JsonSchema,
            input_filename="example.json",
            output=output,
            output_model_type=DataModelType.PydanticV2BaseModel,
        )
        spec = importlib.util.spec_from_file_location(module_name, output)
        module = importlib.util.module_from_spec(spec)
        sys.modules[module_name] = module
        spec.loader.exec_module(module)
    return getattr(module, load)


schema = {
    "title": "Test",
    "type": "object",
    "properties": {
        "a_int": {"default": 1, "title": "A Int", "type": "integer"},
        "i_duration": {
            "default": "PT2H33M3S",
            "format": "duration",
            "title": "I Duration",
            "type": "string",
        },
    },
}

Model = pydantic_model_from_json_schema(schema)
Model.model_fields

/home/jovyan/miniforge3/envs/xlsxdatagrid-dev/lib/python3.12/site-packages/datamodel_code_generator/parser/jsonschema.py:338: UserWarning: format of 'duration' not understood for 'string' - using default
  warn(f'format of {format__!r} not understood for {type_!r} - using default' '')


{'a_int': FieldInfo(annotation=Union[int, NoneType], required=False, default=1, title='A Int'),
 'i_duration': FieldInfo(annotation=Union[str, NoneType], required=False, default='PT2H33M3S', title='I Duration')}

In [15]:
from datetime import timedelta

from pydantic import create_model


def get_timedelta_fields(schema: dict) -> list[str]:
    pr = schema["properties"]
    return [k for k, v in pr.items() if "format" in v and v["format"] == "duration"]


def update_timedelta_field(model: BaseModel, timedelta_fields: list[str]) -> BaseModel:
    """returns a new pydantic model where serialization validators have been added to dates,
    datetimes and durations for compatibility with excel"""

    def get_default(obj):
        return obj.default if hasattr(obj, "default") else ...

    deltas = {
        k: (timedelta, get_default(v))
        for k, v in model.model_fields.items()
        if k in timedelta_fields
    } | {"__base__": model}
    return create_model(model.__name__ + "New", **deltas)


li = get_timedelta_fields(schema)
Model1 = update_timedelta_field(Model, li)
Model1.model_fields

{'a_int': FieldInfo(annotation=Union[int, NoneType], required=False, default=1, title='A Int'),
 'i_duration': FieldInfo(annotation=timedelta, required=False, default='PT2H33M3S')}